In [1]:
from math import *
import numpy as np
import numpy.random as rand
import matplotlib.pyplot as plt
import pandas as pd
import scipy.linalg as lin
import itertools
import ast
from operator import mul
from functools import reduce

In [2]:
Values = pd.DataFrame(columns=['State', 'Value'])

In [3]:
## State to number
def list2int(intList):
    s = ''.join(map(str, intList))
    return int(s)
def state2code(State):
    state_list = [int(item) for row in State.tolist() for item in row]
    state_code = list2int(state_list)
    return state_code

## Number to State
def code2state(code):
    padding = np.zeros(9 - len(str(code)))
    padding = [int(item) for item in padding]
    int2list = padding + [int(x) for x in str(code)]
    code_state = np.array(int2list).reshape((3,3))
    return code_state


In [4]:
## Win or Loss or Neither still
def status(State):
    win = False
    loss = False
    row1 = reduce(mul, State[:,0])
    row2 = reduce(mul, State[:,1])
    row3 = reduce(mul, State[:,2])
    col1 = reduce(mul, State[0,:])
    col2 = reduce(mul, State[1,:])
    col3 = reduce(mul, State[2,:])
    diag1 = reduce(mul, [State[i,i] for i in range(3)])
    diag2 = reduce(mul, [State[i,2-i] for i in range(3)])
    
    if 8 in [row1,row2,row3,col1,col2,col3,diag1,diag2]:
        loss = True
    elif 1 in [row1,row2,row3,col1,col2,col3,diag1,diag2]:
        win = True
    return (win,loss)

In [5]:
## X move
def X_move(State, untaken):
    move_ind = np.random.choice(untaken)
    untaken.remove(move_ind)
    
    rand_move = np.zeros(9)
    rand_move[move_ind - 1] = 1
    return (State + rand_move.reshape((3,3)), untaken)

def X_move_greedy(State, untaken):
    best_value = 0
    for i,item in enumerate(untaken):
        move_ind = untaken[i]
        option_move = np.zeros(9)
        option_move[move_ind - 1] = 1
        option_State = State + option_move.reshape((3,3))

        if state2code(option_State) not in Values.index:
            temp_index = state2code(option_State)
            Values.loc[temp_index,['State','Value']] = [code2state(temp_index),.5]

        if best_value <= Values.loc[state2code(option_State),['Value']][0]:
            best_value = Values.loc[state2code(option_State),['Value']][0]
            best_state = option_State
            best_ind = move_ind

    untaken.remove(best_ind)
    return (best_state, untaken)


## O move
def O_move(State, untaken):
    move_ind = np.random.choice(untaken)
    untaken.remove(move_ind)
    
    rand_move = np.zeros(9)
    rand_move[move_ind - 1] = 1
    return (State + 2*rand_move.reshape((3,3)), untaken)


In [50]:
## Play function
def play(exploit_p,alpha,show=False):
    untaken = [1,2,3,4,5,6,7,8,9]
    State = np.array([[ 0,  0,  0],
                      [ 0,  0,  0],
                      [ 0,  0,  0]])

    ## Intialize
    index = state2code(State)
    if index not in Values.index:
        Values.loc[index,['State','Value']] = [code2state(index),.5]
    old_state_Val = Values.loc[index,['Value']]

    ## Play
    win = False
    loss = False
    i = 0
    while win + loss == 0:
        i = i+1

        r = rand.uniform(0,1)
        if exploit_p > r:
            (State, untaken) = X_move_greedy(State, untaken)
        else: (State, untaken) = X_move(State, untaken)
        
        ## Get new State
        new_index = state2code(State)
        if new_index not in Values.index:
            Values.loc[new_index,['State','Value']] = [code2state(new_index),.5]
        (win,loss) = status(State)

        ## Value update
        if loss == True:
            Values.loc[new_index,['State','Value']] = [code2state(new_index),0]
        elif win == True:
            Values.loc[new_index,['State','Value']] = [code2state(new_index),1]
        if i == 5 and win == False:
            loss = True
            Values.loc[new_index,['State','Value']] = [code2state(new_index),0]
        new_state_Val = Values.loc[new_index,['Value']]
        Values.loc[index,['Value']] = old_state_Val + alpha*(new_state_Val - old_state_Val)
        index = new_index
        old_state_Val = Values.loc[index,['Value']]
        if show == True:
            print(code2state(state2code(State)))
        if i == 5:
            break;
        (State, untaken) = O_move(State, untaken)
    return win


In [69]:
##Train
win_counter = 0
for i in range(100):
    win_counter = win_counter + play(.8,.2)
win_counter

96

In [70]:
##Test
win_counter = 0
for i in range(100):
    win_counter = win_counter + play(1,.1)
win_counter

99

In [73]:
## Play a game
print("Win?",play(1,.1,show = True))

[[0 0 0]
 [0 1 0]
 [0 0 0]]
[[0 0 0]
 [0 1 1]
 [0 0 2]]
[[2 0 0]
 [1 1 1]
 [0 0 2]]
Win? True


In [74]:
Values

,State,Value
0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0]]",0.973008
100000000,"[[1, 0, 0], [0, 0, 0], [0, 0, 0]]",0.807171
10000000,"[[0, 1, 0], [0, 0, 0], [0, 0, 0]]",0.785441
1000000,"[[0, 0, 1], [0, 0, 0], [0, 0, 0]]",0.824125
100000,"[[0, 0, 0], [1, 0, 0], [0, 0, 0]]",0.807484
10000,"[[0, 0, 0], [0, 1, 0], [0, 0, 0]]",0.977095
1000,"[[0, 0, 0], [0, 0, 1], [0, 0, 0]]",0.766323
100,"[[0, 0, 0], [0, 0, 0], [1, 0, 0]]",0.874795
10,"[[0, 0, 0], [0, 0, 0], [0, 1, 0]]",0.779199
1,"[[0, 0, 0], [0, 0, 0], [0, 0, 1]]",0.731312
